<a href="https://colab.research.google.com/github/hila-chefer/Transformer-Explainability/blob/main/BERT_explainability.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from BERT_explainability.modules.BERT.ExplanationGenerator import Generator
from BERT_explainability.modules.BERT.BertForSequenceClassification import BertForSequenceClassification
from transformers import AutoTokenizer

from captum.attr import visualization
import torch

In [2]:
# Bert로 model 불러오기 

model = BertForSequenceClassification.from_pretrained('klue-bert-epoch-32').to("cuda:0")
model.eval()
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")

# initialize the explanations generator
explanations = Generator(model)

classifications = ["020121","000001","02051","020811","020819"]

In [3]:
### model과 tokenizer 잘 불러와졌는지 확인 (Bert)
import numpy as np

text="야 너희 립스틱 이쁘다 엄마가 성적 3등 올랐다고 사준거야 나 오늘 남자친구랑 데이트 하는데 좀 빌려줘라 안 돼 저번에도 너 빌려 가서 망가트렸잖아 너 많이 컸다 이제 말대꾸도 하냐 그게 아니라 엄마가 백화점 가서 특별히 사 준 건데 그러니까 더 좋네 남자친구한테 잘 보이고 오늘 내가 빌려 갈테니까 그런 줄 알아 어디 가서 꼬바르지 말고 알아서 잘 해라 안 되는데"

model=model.cuda()
model.eval()
model_input=tokenizer.encode_plus(text)
input={k:torch.tensor([v], dtype=torch.long).to("cuda:0") for k, v in model_input.items()} #모델에 들어갈 input
input['labels']=None
output=model(**input)
logits=output[0]
pred=logits.detach().cpu().numpy()
pred_idx=np.argmax(pred, axis=1)

print(pred_idx)

[2]


In [4]:
## 모델 구조 확인 

print(model)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32005, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
      (add1): Add()
      (add2): Add()
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
              (matmul1): MatMul()
              (matmul2): MatMul()
              (softmax): Softmax(dim=-1)
              (add): Add()
              (

In [5]:
## 모델 파라미터 확인 

print(model.state_dict())

OrderedDict([('bert.embeddings.position_ids', tensor([[  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
          14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
          28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,
          42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,
          56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,
          70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,
          84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,
          98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111,
         112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125,
         126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139,
         140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153,
         154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167,
      

## practice

In [10]:
# encode a sentence
text_batch = ["야 너희 립스틱 이쁘다 엄마가 성적 3등 올랐다고 사준거야 나 오늘 남자친구랑 데이트 하는데 좀 빌려줘라 안 돼 저번에도 너 빌려 가서 망가트렸잖아 너 많이 컸다 이제 말대꾸도 하냐 그게 아니라 엄마가 백화점 가서 특별히 사 준 건데 그러니까 더 좋네 남자친구한테 잘 보이고 오늘 내가 빌려 갈테니까 그런 줄 알아 어디 가서 꼬바르지 말고 알아서 잘 해라 안 되는데"]
encoding = tokenizer(text_batch, return_tensors='pt')
input_ids = encoding['input_ids'].to("cuda:0")
attention_mask = encoding['attention_mask'].to("cuda:0")

In [11]:
# generate an explanation for the input
expl = explanations.generate_LRP(input_ids=input_ids, attention_mask=attention_mask, start_layer=0)[0]
# normalize scores
expl = (expl - expl.min()) / (expl.max() - expl.min())

In [14]:
# get the model classification
output = torch.nn.functional.softmax(model(input_ids=input_ids, attention_mask=attention_mask)[0], dim=-1)
classification = output.argmax(dim=-1).item()

# get class name
class_name = classifications[classification]

# if the classification is negative, higher explanation scores are more negative
# flip for visualization
#if class_name == "NEGATIVE":
  #expl *= (-1)

tokens = tokenizer.convert_ids_to_tokens(input_ids.flatten())
true_class = 2

print([(tokens[i], expl[i].item()) for i in range(len(tokens))])

vis_data_records = [visualization.VisualizationDataRecord(
                                expl,
                                output[0][classification],
                                classification,
                                true_class,
                                true_class,
                                1,       
                                tokens,
                                1)]
visualization.visualize_text(vis_data_records)

[('[CLS]', 0.0), ('야', 0.15335261821746826), ('너희', 0.05094880610704422), ('립스틱', 0.046912260353565216), ('이쁘', 0.016315625980496407), ('##다', 0.008059415966272354), ('엄마', 0.036335889250040054), ('##가', 0.00039619702147319913), ('성적', 0.003914471250027418), ('3', 0.0008176201372407377), ('##등', 0.0013208386953920126), ('올랐', 0.0016785404877737164), ('##다고', 0.0054642558097839355), ('사', 0.08747601509094238), ('##준', 0.0330142080783844), ('##거', 0.14503535628318787), ('##야', 0.2392244040966034), ('나', 0.00792168639600277), ('오늘', 0.0045907143503427505), ('남자', 0.0007658209069631994), ('##친', 0.004305955953896046), ('##구', 0.0), ('##랑', 0.004645790439099073), ('데이트', 0.02038019336760044), ('하', 0.02216663770377636), ('##는데', 0.0030992566607892513), ('좀', 0.8331661224365234), ('빌려', 1.0), ('##줘', 0.44333335757255554), ('##라', 0.35211116075515747), ('안', 0.4230926036834717), ('돼', 0.4184347093105316), ('저번', 0.07143042981624603), ('##에도', 0.021028349176049232), ('너', 0.17330843210220337),

## test version

In [6]:
# encode a sentence
text_batch = ["안녕하세요 넌 지금 내가 안녕한걸로 보이냐 아침부터 왜 그러세요 하 너 진짜 꼴보기 싫어 죽이고 싶어 자꾸 이러시면 신고합니다 뭐 신고를 해 그래 어디 신고해 봐 신고하기 전에 죽여버릴테니까 내가 당신 때문에 얼마나 괴로운 줄 알아 너만 괴롭냐 나도 너 때문에 괴로워 네가 살아있는 게 괴로워 뭐라고요 그럼 내가 죽어야 돼요 어 넌 죽어야 돼 내가 너 죽일 거니까 기다려 지금 죽인다고 협박 하는 거예요 그래 죽인다고 죽여 버린다고 야 이거 완전 미친놈 아냐 당장 이리로 와 모가지 분질러 버릴테 싫어 하지 마 끝까지 따라가서 죽일 거니까 도망 갈 생각하지 말아라"]
encoding = tokenizer(text_batch, return_tensors='pt')
input_ids = encoding['input_ids'].to("cuda:0")
attention_mask = encoding['attention_mask'].to("cuda:0")

# generate an explanation for the input
expl = explanations.generate_LRP(input_ids=input_ids, attention_mask=attention_mask, start_layer=0)[0]
# normalize scores
expl = (expl - expl.min()) / (expl.max() - expl.min())

# get the model classification
output = torch.nn.functional.softmax(model(input_ids=input_ids, attention_mask=attention_mask)[0], dim=-1)
classification = output.argmax(dim=-1).item()

# get class name
class_name = classifications[classification]

# if the classification is negative, higher explanation scores are more negative
# flip for visualization
#if class_name == "NEGATIVE":
  #expl *= (-1)

tokens = tokenizer.convert_ids_to_tokens(input_ids.flatten())
true_class = 0

print([(tokens[i], expl[i].item()) for i in range(len(tokens))])

vis_data_records = [visualization.VisualizationDataRecord(
                                expl,
                                output[0][classification],
                                classification,
                                true_class,
                                true_class,
                                1,       
                                tokens,
                                1)]
visualization.visualize_text(vis_data_records)

[('[CLS]', 0.0), ('안녕', 0.01822124980390072), ('##하', 0.0005918995011597872), ('##세요', 0.09357145428657532), ('넌', 0.008544906042516232), ('지금', 0.003679894842207432), ('내', 0.012274110689759254), ('##가', 0.0007257859106175601), ('안녕', 0.0006193562876433134), ('##한', 0.0), ('##걸', 0.005222924053668976), ('##로', 0.0016151074087247252), ('보이', 0.008212365210056305), ('##냐', 0.00874069333076477), ('아침', 0.008741514757275581), ('##부터', 0.0016493494622409344), ('왜', 0.012132026255130768), ('그러', 0.009389461018145084), ('##세요', 0.024447839707136154), ('하', 0.002954364288598299), ('너', 0.015305454842746258), ('진짜', 0.026133384555578232), ('꼴', 0.1540972739458084), ('##보기', 0.005195071455091238), ('싫어', 0.08452524244785309), ('죽이', 1.0), ('##고', 0.19279015064239502), ('싶', 0.05834188684821129), ('##어', 0.012064863927662373), ('자꾸', 0.007165264803916216), ('이러', 0.0066926488652825356), ('##시', 0.00823504664003849), ('##면', 0.00579339312389493), ('신고', 0.013876495882868767), ('##합니다', 0.05211871

In [7]:
# encode a sentence
text_batch = ["어이 아저씨 편의점에서 방금 담배 샀죠 그거 한 대만 줘봐요 예 담배 한 대만 달라고요 지금 고등학생 아니에요 담배 한대 달라는데 고등학생이 무슨 상관이에요 아저씬 늙어서 폈어요 무슨 또 한 대만 줘요 다 내놔요 학생들 이러면 안 돼요 아저씨 지금 넓은 데서 얘기 하니까 우리가 사람 좋아 보여요 아닙니다 아저씨 돈 얼마 있어요 저 아저씨 아니고 백수라서 돈도 없어요 백수가 돈 없이 담배를 어떻게 펴요 5만 원도 없어요 없어요 안 맞으니까 돈이 안 나오지 좋은 말할 때 다시 편의점 들어가서 담배 한 보루 만 사 가지고 와요 허튼 짓 하지 마요 우리 여기 동네 살아서 아저씨 어디 사는지 다 지켜볼 거야 알겠어요 담배 뭐로 사다 주면 되죠 야 아무거나 사 와"]
encoding = tokenizer(text_batch, return_tensors='pt')
input_ids = encoding['input_ids'].to("cuda:0")
attention_mask = encoding['attention_mask'].to("cuda:0")

# generate an explanation for the input
expl = explanations.generate_LRP(input_ids=input_ids, attention_mask=attention_mask, start_layer=0)[0]
# normalize scores
expl = (expl - expl.min()) / (expl.max() - expl.min())

# get the model classification
output = torch.nn.functional.softmax(model(input_ids=input_ids, attention_mask=attention_mask)[0], dim=-1)
classification = output.argmax(dim=-1).item()

# get class name
class_name = classifications[classification]

# if the classification is negative, higher explanation scores are more negative
# flip for visualization
#if class_name == "NEGATIVE":
  #expl *= (-1)

tokens = tokenizer.convert_ids_to_tokens(input_ids.flatten())
true_class = 3

print([(tokens[i], expl[i].item()) for i in range(len(tokens))])

vis_data_records = [visualization.VisualizationDataRecord(
                                expl,
                                output[0][classification],
                                classification,
                                true_class,
                                true_class,
                                1,       
                                tokens,
                                1)]
visualization.visualize_text(vis_data_records)

[('[CLS]', 0.0), ('어이', 0.03984961286187172), ('아저씨', 0.07450990378856659), ('편의점', 0.01882506161928177), ('##에서', 0.003192710690200329), ('방금', 0.007001399993896484), ('담배', 0.009717537090182304), ('샀', 0.031038004904985428), ('##죠', 0.014797032810747623), ('그거', 0.04449158161878586), ('한', 0.004086912609636784), ('대만', 0.0265947338193655), ('줘', 0.3070613145828247), ('##봐', 0.043368130922317505), ('##요', 0.011637110263109207), ('예', 0.13262352347373962), ('담배', 0.013128753751516342), ('한', 0.006977537181228399), ('대만', 0.04725651443004608), ('달라', 0.46646881103515625), ('##고', 0.028360309079289436), ('##요', 0.016956321895122528), ('지금', 0.0008265360374934971), ('고등학생', 0.0017246633069589734), ('아니', 0.0014698567101731896), ('##에', 0.002752503613010049), ('##요', 0.011365226469933987), ('담배', 0.0036084435414522886), ('한대', 0.0077188932336866856), ('달라', 0.15417678654193878), ('##는데', 0.005083549302071333), ('고등학생', 0.000951005204115063), ('##이', 0.0010374330449849367), ('무슨', 0.0026172

In [11]:
# encode a sentence
text_batch = ["김 대리 네 과장님 부르셨습니까 네 이 옷 이 꼬라지가 이 뭐냐 예 제 옷에 무슨 문제라도 있습니까 너 지금 그걸 몰라서 묻는 거냐 일을 못하면 센스라도 있어야지 분위기 파악을 못 해서야 너 지금 회사 출근하는 복장이라고 생각하냐 꼬질꼬질해가지고는 네 마누라는 집에서 이런 것도 안 챙겨 주고 뭐 한데 과장님 김대리 요즘 집사람하고 사이가 안 좋다고 합니다 그렇지 집안도 제대로 건사하지 못하는게 회사 일이라고 똑부러지게 하겠냐 죄송합니다 오늘 아침부터 김대리 보니까 기분이 팍 상하네"]
encoding = tokenizer(text_batch, return_tensors='pt')
input_ids = encoding['input_ids'].to("cuda:0")
attention_mask = encoding['attention_mask'].to("cuda:0")

# generate an explanation for the input
expl = explanations.generate_LRP(input_ids=input_ids, attention_mask=attention_mask, start_layer=0)[0]
# normalize scores
expl = (expl - expl.min()) / (expl.max() - expl.min())

# get the model classification
output = torch.nn.functional.softmax(model(input_ids=input_ids, attention_mask=attention_mask)[0], dim=-1)
classification = output.argmax(dim=-1).item()

# get class name
class_name = classifications[classification]

# if the classification is negative, higher explanation scores are more negative
# flip for visualization
#if class_name == "NEGATIVE":
  #expl *= (-1)

tokens = tokenizer.convert_ids_to_tokens(input_ids.flatten())
true_class = 3

print([(tokens[i], expl[i].item()) for i in range(len(tokens))])

vis_data_records = [visualization.VisualizationDataRecord(
                                expl,
                                output[0][classification],
                                classification,
                                true_class,
                                true_class,
                                1,       
                                tokens,
                                1)]
visualization.visualize_text(vis_data_records)

[('[CLS]', 0.0), ('김', 0.0783056989312172), ('대리', 0.29976680874824524), ('네', 0.14213532209396362), ('과장', 0.17372332513332367), ('##님', 0.010379365645349026), ('부르', 0.017262661829590797), ('##셨', 0.000820184126496315), ('##습니까', 0.019010374322533607), ('네', 0.18617680668830872), ('이', 0.009316799230873585), ('옷', 0.027230555191636086), ('이', 0.004727375227957964), ('꼬', 0.0029416517354547977), ('##라지', 0.0014689741656184196), ('##가', 0.008201261050999165), ('이', 0.0038032126612961292), ('뭐', 0.0007677284302189946), ('##냐', 0.0033139665611088276), ('예', 0.22777687013149261), ('제', 0.06004992499947548), ('옷', 0.004449774045497179), ('##에', 0.0009788109455257654), ('무슨', 0.00016231602057814598), ('문제', 0.0), ('##라도', 0.000803235569037497), ('있', 0.000738060858566314), ('##습니까', 0.020105965435504913), ('너', 0.058004532009363174), ('지금', 0.010379079729318619), ('그걸', 0.0035035342443734407), ('몰라서', 0.0009122506598941982), ('묻', 0.004527953919023275), ('##는', 0.003700974164530635), ('거', 

In [8]:
# encode a sentence
text_batch = ["그만 때려 왜 때려 나도 몰라 돼지 새끼들은 패는 맛이 있거든 난 돼지가 아니야 넌 좀 더 맞아야 되겠어 영재야 암바 이렇게 하는 거였냐 아아 아파 재밌지 개 재밌어 보이네 뭐 하냐 나도 좀 끼자 암바 연습하고 있어 이 새끼 이거 암바 못봤냐 암바는 이게 암바지 병신아 오 그래 영재야 딜로틴 초크도 가능하냐 한 번도 안 해 봤는데 오늘 각이다 씁 이렇게 걸어서 제대로 조이면 기절까지 5초컷 인정 어 인정 제발 그만해"]
encoding = tokenizer(text_batch, return_tensors='pt')
input_ids = encoding['input_ids'].to("cuda:0")
attention_mask = encoding['attention_mask'].to("cuda:0")

# generate an explanation for the input
expl = explanations.generate_LRP(input_ids=input_ids, attention_mask=attention_mask, start_layer=0)[0]
# normalize scores
expl = (expl - expl.min()) / (expl.max() - expl.min())

# get the model classification
output = torch.nn.functional.softmax(model(input_ids=input_ids, attention_mask=attention_mask)[0], dim=-1)
classification = output.argmax(dim=-1).item()

# get class name
class_name = classifications[classification]

# if the classification is negative, higher explanation scores are more negative
# flip for visualization
#if class_name == "NEGATIVE":
  #expl *= (-1)

tokens = tokenizer.convert_ids_to_tokens(input_ids.flatten())
true_class = 3

print([(tokens[i], expl[i].item()) for i in range(len(tokens))])

vis_data_records = [visualization.VisualizationDataRecord(
                                expl,
                                output[0][classification],
                                classification,
                                true_class,
                                true_class,
                                1,       
                                tokens,
                                1)]
visualization.visualize_text(vis_data_records)

[('[CLS]', 0.0), ('그만', 0.1544756442308426), ('때려', 0.406770259141922), ('왜', 0.012310229241847992), ('때려', 0.6146738529205322), ('나', 0.009066042490303516), ('##도', 0.007705036550760269), ('몰라', 0.030198875814676285), ('돼지', 0.1263611763715744), ('새끼', 0.04302041605114937), ('##들', 0.017994048073887825), ('##은', 0.03365359827876091), ('패', 0.08901943266391754), ('##는', 0.005003277212381363), ('맛', 0.011124507524073124), ('##이', 0.0), ('있', 0.011188912205398083), ('##거', 0.01691933535039425), ('##든', 0.08677534759044647), ('난', 0.04089942201972008), ('돼지', 0.07346087694168091), ('##가', 0.008674997836351395), ('아니야', 0.03306010365486145), ('넌', 0.20058105885982513), ('좀', 0.0017852005548775196), ('더', 0.05232767388224602), ('맞아', 0.07947655022144318), ('##야', 0.011407475918531418), ('되', 0.030018039047718048), ('##겠', 0.029100770130753517), ('##어', 0.12042349576950073), ('영재', 0.7668342590332031), ('##야', 0.17312553524971008), ('암', 0.10928467661142349), ('##바', 0.13735055923461914), ('

In [9]:
# encode a sentence
text_batch = ["잡았다 오 정말 뭐 잡았어 우럭이다 히히 이놈 엄청 큰데 대박이다 야 당장 회떠먹자 근데 이거 어떻게 죽여야 되지 어 거기 대가리 위쪽을 칼로 쑤셔 가지고 단숨에 숨통을 끊어야 돼 대가리 어디 정확하게 말해 봐 거기 있잖아 정수리 보다 2cm 위쪽이랑 몸통이랑 이어지는 부분 아 여기 여기 자르면 바로 죽일 수 있어 어 거기 거기서 해서 죽여봐 죽었나 죽은 거 같은데 그럼 이제 피 빼야겠다 어 거꾸로 놓고 흐르는 물에 피 빼봐 내장도 다 빼야 되나 당연하지 어 그래 내장도 전부 다 뺐어 그러면 이제 비늘을 제거 해봐 아이 깜짝이야 얘 죽었는데 지금 꿈틀 걸었어 아 원래 그래 근육이 움직이는 거야"]
encoding = tokenizer(text_batch, return_tensors='pt')
input_ids = encoding['input_ids'].to("cuda:0")
attention_mask = encoding['attention_mask'].to("cuda:0")

# generate an explanation for the input
expl = explanations.generate_LRP(input_ids=input_ids, attention_mask=attention_mask, start_layer=0)[0]
# normalize scores
expl = (expl - expl.min()) / (expl.max() - expl.min())

# get the model classification
output = torch.nn.functional.softmax(model(input_ids=input_ids, attention_mask=attention_mask)[0], dim=-1)
classification = output.argmax(dim=-1).item()

# get class name
class_name = classifications[classification]

# if the classification is negative, higher explanation scores are more negative
# flip for visualization
#if class_name == "NEGATIVE":
  #expl *= (-1)

tokens = tokenizer.convert_ids_to_tokens(input_ids.flatten())
true_class = 3

print([(tokens[i], expl[i].item()) for i in range(len(tokens))])

vis_data_records = [visualization.VisualizationDataRecord(
                                expl,
                                output[0][classification],
                                classification,
                                true_class,
                                true_class,
                                1,       
                                tokens,
                                1)]
visualization.visualize_text(vis_data_records)

[('[CLS]', 0.0), ('잡', 0.07071039080619812), ('##았', 0.009943670593202114), ('##다', 0.14060881733894348), ('오', 0.22511810064315796), ('정말', 0.06455623358488083), ('뭐', 0.11283937841653824), ('잡', 0.03423244133591652), ('##았', 0.009106727316975594), ('##어', 0.19638238847255707), ('우', 0.113436259329319), ('##럭', 0.16495133936405182), ('##이다', 0.14047864079475403), ('히히', 0.26865360140800476), ('이놈', 0.16402243077754974), ('엄청', 0.020999053493142128), ('큰', 0.09631840884685516), ('##데', 0.011786368675529957), ('대박', 0.12194567173719406), ('##이다', 0.5836271047592163), ('야', 0.6053787469863892), ('당장', 0.0076998076401650906), ('회', 0.1970745027065277), ('##떠', 0.019342003390192986), ('##먹', 0.016515439376235008), ('##자', 0.08131138235330582), ('근데', 0.08647583425045013), ('이거', 0.17970234155654907), ('어떻게', 0.007697241846472025), ('죽여', 0.13779529929161072), ('##야', 0.020385392010211945), ('되', 0.026141759008169174), ('##지', 0.28690120577812195), ('어', 0.7093088626861572), ('거기', 0.101061